In [ ]:
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
import gym
import or_gym

###################################################
# Deep Deterministic Policy Gradient
###################################################

class Actor(nn.Module):
    def __init__(self, dim_state, dim_action, fc1, fc2, alpha,checkpoint='actor' ):
        super(Actor, self).__init__()
        self.dim_state = dim_state
        self.dim_action = dim_action
        
        self.linear1 = nn.Linear(dim_state, fc1)
        self.linear2 = nn.Linear(fc1, fc2)
        self.linear3 = nn.Linear(fc2, dim_action)
        
        self.device = t.device('cpu' if t.cuda.is_available() else 'cpu')
        self.optimizer = optim.Adam(self.parameters(), lr=alpha, weight_decay=1e-4)
        self.to(self.device)
        self.path = checkpoint
        
    def forward(self, state):
        output = state.to(self.device)
        
        output = f.relu(self.linear1(output))
        output = f.relu(self.linear2(output))
        output = self.linear3(output)
        
        return output
    
    def saveCheckpoint(self):
        print ('...saving check point...')
        t.save(self.state_dict(), self.path)
    
    def loadCheckpoint(self):
        print('...loading check point')
        self.load_state_dict(t.load(self.path))
    
class Critic(nn.Module):   
    def __init__(self, dim_state, dim_action, fc1, fc2, alpha,checkpoint='critic'):
        super(Critic, self).__init__()
        self.dim_state = dim_state
        self.dim_action = dim_action
        self.path = checkpoint
        
        self.linear1 = nn.Linear(dim_state+ dim_action, fc1)
        self.linear2 = nn.Linear(fc1, fc2)
        self.linear3 = nn.Linear(fc2,1)
        self.optimizer = optim.Adam(self.parameters(), lr = alpha, weight_decay=1e-4)
        self.device = t.device('cpu' if t.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, state, action):
        state = state.to(self.device)
        action = action.to(self.device)
        
        output = t.relu(self.linear1(t.cat((state, action), dim=1)))
        output = t.relu(self.linear2(output))
        output = self.linear3(output)
        
        return output
    
    def saveCheckpoint(self):
        print('...save checkout...')
        t.save(self.state_dict(), self.path)
    
    def loadCheckpoint(self):
        print ('...load checj point...')
        self.load_state_dict(t.load(self.path))
        
        
class replayBuffer():
    def __init__(self, maxm_size, dim_state, dim_action):
        self.counter = 0
        self.state_mem = np.zeros((maxm_size, dim_state))
        self.action_mem = np.zeros((maxm_size, dim_action))
        self.reward_mem = np.zeros(maxm_size)
        self.state_new_mem = np.zeros((maxm_size, dim_state))
        self.done_mem = np.zeros(maxm_size)
        self.maxm_size = maxm_size
        
    def store_Transaction(self, state, action, reward, state_new, done):
        index = self.counter % self.maxm_size
        
        self.state_mem[index] = state
        self.action_mem[index] = action
        self.reward_mem[index] = reward
        self.state_new_mem[index] =state_new
        self.done_mem[index] = 1.0 - done
        
        self.counter+=1
    
    def sample_batch(self, batch_size=126):
        maxm_size = min(self.counter, self.maxm_size)
        batch = np.random.choice(maxm_size, batch_size)
        
        state_batch = self.state_mem[batch]
        action_batch = self.action_mem[batch]
        reward_batch = self.reward_mem[batch]
        state_new_batch = self.state_new_mem[batch]
        done_batch = self.done_mem[batch]
        
        return state_batch, action_batch,reward_batch,state_new_batch,done_batch
        
    
class Agent():
    def __init__(self, env, dim_state, dim_action, fc1, fc2, alpha, gamma, maxm_size, maxm_iters=50, batch_size=126):
        self.dim_state = dim_state
        self.dim_action = dim_action
        self.maxm_iters = maxm_iters
        self.actor=Actor(dim_state, dim_action, fc1, fc2, alpha,checkpoint='actor')
        self.critic=Critic(dim_state, dim_action, fc1, fc2, alpha,checkpoint='critic')
        self.buffer = replayBuffer(maxm_size, dim_state, dim_action) # maxm_size, dim_state, dim_action):
        self.gamma = gamma
        self.batch_size = batch_size
        
        self.device = t.device('cpu' if t.cuda.is_available() else 'cpu')
        
    def plot_learning_curve(self,x, scores, figure_file='test'):
        running_avg = np.zeros(len(scores))
        for i in range(len(running_avg)):
            running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
        plt.plot(x, running_avg)
        plt.title('Running average of previous 100 scores')
        #plt.savefig(figure_file)
        
    
    def train(self):
        
        self.critic.train()
        self.actor.train()
        reward_list = []  
           
        for i in range(self.maxm_iters):
            self.critic.zero_grad()
            self.actor.zero_grad()
            
            done= False
            state = env.reset()       
            total_reward= 0
            
            if not done:
                action = self.actor(t.tensor(state).float().to(self.device)).detach().numpy()
                state_new, reward, done, _ = env.step(action)
                total_reward+=reward
        
                self.buffer.store_Transaction(state, action, reward, state_new, done)
                   
                if self.buffer.counter > self.batch_size:
                    state_batch, action_batch, reward_batch, state_new_batch, done_batch = self.buffer.sample_batch()
                        
                    ##Update the critic
                    target = t.tensor(reward_batch).view(-1,1)+self.gamma*self.critic(t.tensor(state_new_batch).float(),self.actor(t.tensor(state_new_batch).float()))*t.tensor(done_batch).view(-1,1)
                    preds = self.critic(t.tensor(state_batch).float(), t.tensor(action_batch).float())
        
                    loss_critic = t.sum((target - preds)**2)
                    loss_critic.backward()
                    self.critic.optimizer.step()
        
                    ##Update the actor
                    loss_actor = -self.critic(t.tensor(state_batch).float(), self.actor(t.tensor(state_batch).float()))
                    loss_actor = loss_actor.mean()
                    loss_actor.backward()
                    self.actor.optimizer.step()
        
                    state = state_new
            reward_list.append(total_reward)
        #print (reward_list)
        
        x = [i+1 for i in range(self.maxm_iters)]
        self.plot_learning_curve(x, reward_list)
        
if __name__ == '__main__':
    env = gym.make('Pendulum-v0')
    #env = or_gym.make('InvManagement-v1')
    
    dim_state = env.observation_space.shape[0]
    dim_action = env.action_space.shape[0]
    agent = Agent(env,3,1, 256, 256, 1e-5, 0.99, 3000, maxm_iters=1000)
    agent.train()
    
    


In [4]:
for i in range(10):
    #print(env.observation_space.sample())
    print(env.action_space.sample())

[1.6032397]
[-1.7046111]
[1.548891]
[-1.9472643]
[-0.2563781]
[-1.333235]
[-0.6761971]
[-0.2355361]
[-1.3476692]
[-0.22001511]
